In [1]:
import pandas as pd
import os

def get_image_path(image_id:int):
    return os.path.join('../tiles_2048', str(image_id))

I_FOLD = 0
train = pd.read_csv(f"train_fold_{I_FOLD}.csv")
validation = pd.read_csv(f"val_fold_{I_FOLD}.csv")

train['tile_path'] = train['image_id'].apply(lambda x: get_image_path(x))
validation['tile_path'] = validation['image_id'].apply(lambda x: get_image_path(x))
train.head()

,image_id,label,image_width,image_height,is_tma,tile_path
0,8280,HGSC,2964,2964,True,../tiles_2048/8280
1,8985,LGSC,65003,31754,False,../tiles_2048/8985
2,9183,LGSC,74091,34185,False,../tiles_2048/9183
3,9200,MC,3388,3388,True,../tiles_2048/9200
4,10252,LGSC,49053,39794,False,../tiles_2048/10252


In [2]:
from PIL import Image
import torch
import torch.nn as nn
import timm
from timm.models.layers import DropPath
import copy

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "timm/eva02_base_patch14_448.mim_in22k_ft_in22k_in1k"

print(f"Using device {device} and model {model_name}")

model = timm.create_model(model_name, pretrained=True)

# Define the maximum drop path rate
"""
# for folds 
max_drop_path_rate = 0.3
dropout_rate = 0.1
"""
# for folds 0
max_drop_path_rate = 0.15
dropout_rate = 0.0

drop_path_rates = [x.item() for x in torch.linspace(0, max_drop_path_rate, len(model.blocks))]

# Assign drop path rates
for i, block in enumerate(model.blocks):
    block.drop_path1 = DropPath(drop_prob=drop_path_rates[i])
    block.drop_path2 = DropPath(drop_prob=drop_path_rates[i])
    block.attn.attn_drop = nn.Dropout(p=dropout_rate, inplace=False)
    block.attn.proj_drop = nn.Dropout(p=dropout_rate, inplace=False)
    block.mlp.drop1 = nn.Dropout(p=dropout_rate, inplace=False)
    block.mlp.drop2 = nn.Dropout(p=dropout_rate, inplace=False)

model.head = nn.Linear(model.head.in_features, 5)

model = model.to(device)

# Initialize EMA model
ema_decay = 0.999  # decay factor for EMA
ema_model = copy.deepcopy(model)
ema_model = ema_model.to(device)

Using device cuda and model timm/eva02_base_patch14_448.mim_in22k_ft_in22k_in1k


/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
import os
from PIL import Image
from torch.utils.data import Dataset
import random

integer_to_label = {
    0: 'HGSC',
    1: 'CC',
    2: 'EC',
    3: 'LGSC',
    4: 'MC',
}

label_to_integer = {
    'HGSC': 0,
    'CC': 1,
    'EC': 2,
    'LGSC': 3,
    'MC': 4,
}

class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.all_images = []  # Store all images in an interlaced fashion
        self.folder_images = []  # Temporary storage for images from each folder

        # Step 1: Collect all images from each folder
        for index, row in dataframe.iterrows():
            folder_path = row['tile_path']
            label = row['label']
            image_id = row['image_id']
            if os.path.isdir(folder_path):
                image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.png')]
                random.shuffle(image_files)
                self.folder_images.append([(image_file, label, image_id) for image_file in image_files])

        # Step 2: Interlace the images
        max_length = max(len(images) for images in self.folder_images)
        for i in range(max_length):
            for images in self.folder_images:
                self.all_images.append(images[i % len(images)])

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        image_path, label, image_id = self.all_images[idx]
        image = Image.open(image_path)
        
        if self.transform:
            image = self.transform(image)

        return image, label_to_integer[label], image_id

In [4]:
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms

BATCH_SIZE = 8

train_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine(degrees=45, translate=(0.25, 0.25), scale=(1, 2), shear=(-30, 30, -30, 30)),
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(mean=[
        0.48145466,
        0.4578275,
        0.40821073
    ], std=[
        0.26862954,
        0.26130258,
        0.27577711
    ]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
])

val_transform = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(mean=[
        0.48145466,
        0.4578275,
        0.40821073
    ], std=[
        0.26862954,
        0.26130258,
        0.27577711
    ]),
])

train_dataset = ImageDataset(dataframe=train, transform=train_transform)

# Calculate weights for each class
class_counts = [train.groupby('label').count().loc[label]['image_id'] for label in label_to_integer]  # Example class counts
num_samples = sum(class_counts)
class_weights = [num_samples / class_count for class_count in class_counts]

# Assign a weight to each sample in the dataset based on its class
sample_weights = [class_weights[label_to_integer[label]] for _, label, _ in train_dataset.all_images]

# Create WeightedRandomSampler
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoader with WeightedRandomSampler
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=3)

In [5]:
import logging
import sys

# Get the root logger
logger = logging.getLogger()

# Optional: Remove all existing handlers from the logger
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# Set the logging level
logger.setLevel(logging.INFO)

# Create a FileHandler and add it to the logger
file_handler = logging.FileHandler(f'logs/eva02_base_2048/fold_{I_FOLD}.txt')
file_handler.setLevel(logging.INFO)
logger.addHandler(file_handler)

# Create a StreamHandler for stderr and add it to the logger
stream_handler = logging.StreamHandler(sys.stderr)
stream_handler.setLevel(logging.ERROR)  # Only log ERROR and CRITICAL messages to stderr
logger.addHandler(stream_handler)

In [ ]:
import torch
import torch.optim as optim
import logging
import numpy as np
import math
from sklearn.metrics import balanced_accuracy_score
import random
from torch.cuda.amp import GradScaler, autocast

initial_lr = 1e-5
final_lr = 1e-6
num_epochs = 10000

# Function for linear warmup
def learning_rate(step, warmup_steps=2000, max_steps=20000):
    if step < warmup_steps:
        return initial_lr * (float(step) / float(max(1, warmup_steps)))
    elif step < max_steps:
        progress = (float(step - warmup_steps) / float(max(1, max_steps - warmup_steps)))
        cos_component = 0.5 * (1 + math.cos(math.pi * progress))
        return final_lr + (initial_lr - final_lr) * cos_component
    else:
        return final_lr

# Function to calculate weighted accuracy
def weighted_accuracy(true_labels, predictions, class_weights):
    correct = 0
    total_weight = 0

    for label, pred in zip(true_labels, predictions):
        if label == pred:
            correct += class_weights[label]
        total_weight += class_weights[label]

    return correct / total_weight

def update_ema_variables(model, ema_model, alpha, global_step):
    # Update the EMA model parameters
    with torch.no_grad():
        for ema_param, param in zip(ema_model.parameters(), model.parameters()):
            ema_param.data.mul_(alpha).add_(param.data, alpha=1 - alpha)

scaler = GradScaler()
optimizer = optim.AdamW(model.parameters(), lr=initial_lr, weight_decay=5e-2)

# Define the loss function with class weights
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

best_val_accuracy = 0.0
step = 0

for epoch in range(num_epochs):
    model.train()  # set the model to training mode
    
    for i, (images, labels, _) in enumerate(train_dataloader, 0):
        # Convert images to PIL format
        images = images.to(device)
        labels = labels.to(device)
        
        # Linearly increase the learning rate
        lr = learning_rate(step)
        for g in optimizer.param_groups:
            # g['lr'] = g['lr'] * lr / initial_lr
            g['lr'] = lr

        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass with autocast
        with autocast():
            outputs = model(images)
            logits_per_image = outputs
            loss = criterion(logits_per_image, labels)
        
        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        update_ema_variables(model, ema_model, ema_decay, step)

        logging.info('[%d, %5d] loss: %.3f' % (epoch + 1, step, loss.item()))

        if step % 2000 == 0:
            ema_model.eval()
            torch.save(ema_model.state_dict(), f'eva02_base_models_2048/fold_{I_FOLD}/epoch_{epoch}_step_{step}.pth')
            logging.info(f'Model saved after epoch {epoch} and step {step}')\

            model.train()

        if step == 20000:
            torch.save(ema_model.state_dict(), f'eva02_base_models_2048/fold_{I_FOLD}/final.pth')
            break
    
        step += 1
        
    if step >= 20000:
        break

In [7]:
import random
from sklearn.metrics import balanced_accuracy_score

ema_model.eval()

image_ids = []
logits = []
labels = []

with torch.no_grad():
    for _, row in validation.iterrows():
        path = row['tile_path']
        all_files = [f for f in os.listdir(path) if f.lower().endswith('.png')]

        sum_probabilities = torch.zeros(5).to(device)
        sum_log_probabilities = torch.zeros(5).to(device)
        sum_log_neg_probabilities = torch.zeros(5).to(device)
        
        batch_logits = []

        # Prepare a list to hold image tiles
        batch_tiles = []

        sample_size = min(128, len(all_files))
        sampled_files = random.sample(all_files, sample_size)

        for image_name in sampled_files:
            image_path = os.path.join(path, image_name)
            sub_image = Image.open(image_path)

            tile = val_transform(sub_image).unsqueeze(0).to(device)
            batch_tiles.append(tile)
        
        for i_batch in range(0, len(batch_tiles), 32):
            outputs = ema_model(torch.concat(batch_tiles[i_batch:i_batch+32], dim=0))
            probs = outputs.softmax(dim=1)
            batch_logits.append(outputs)
            sum_probabilities += probs.sum(dim=0)
            sum_log_probabilities += torch.log(probs).sum(dim=0)
            sum_log_neg_probabilities += torch.log(1 - probs).sum(dim=0)
        
        image_id = row['image_id']
        batch_logits = torch.concat(batch_logits, dim=0)
        label = row['label']
        print(batch_logits.shape, label, image_id, integer_to_label[sum_probabilities.argmax().cpu().item()], integer_to_label[sum_log_probabilities.argmax().cpu().item()], integer_to_label[sum_log_neg_probabilities.argmin().cpu().item()])
        image_ids.append(image_id)
        logits.append(batch_logits)
        labels.append(label)
        

torch.Size([80, 5]) HGSC 4 HGSC HGSC HGSC
torch.Size([128, 5]) LGSC 66 HGSC HGSC CC
torch.Size([4, 5]) HGSC 91 HGSC HGSC HGSC
torch.Size([56, 5]) LGSC 281 EC EC EC
torch.Size([128, 5]) EC 286 HGSC HGSC HGSC
torch.Size([128, 5]) HGSC 431 HGSC HGSC HGSC
torch.Size([128, 5]) HGSC 706 HGSC HGSC HGSC
torch.Size([91, 5]) HGSC 970 HGSC HGSC HGSC
torch.Size([128, 5]) HGSC 1020 HGSC HGSC HGSC
torch.Size([102, 5]) HGSC 1080 HGSC HGSC HGSC
torch.Size([69, 5]) HGSC 1101 HGSC HGSC HGSC
torch.Size([128, 5]) HGSC 1252 HGSC HGSC HGSC
torch.Size([128, 5]) HGSC 1289 LGSC HGSC LGSC
torch.Size([95, 5]) HGSC 1295 EC EC EC
torch.Size([128, 5]) CC 1660 CC CC CC
torch.Size([78, 5]) HGSC 1666 HGSC HGSC HGSC
torch.Size([128, 5]) HGSC 1774 CC HGSC CC
torch.Size([128, 5]) HGSC 1925 HGSC HGSC HGSC
torch.Size([128, 5]) CC 1943 EC EC EC
torch.Size([128, 5]) CC 1952 CC CC CC
torch.Size([114, 5]) HGSC 2097 HGSC HGSC HGSC
torch.Size([128, 5]) MC 2227 MC MC MC
torch.Size([128, 5]) LGSC 2391 LGSC LGSC LGSC
torch.Size([12

In [8]:
predictions = []
for image_logits in logits:
    argmax_indices = torch.argmax(image_logits, dim=1)

    frequency_counts = torch.bincount(argmax_indices, minlength=5)

    max_vote_key = frequency_counts.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

plurality_accuracy = balanced_accuracy_score(labels, predictions)
plurality_accuracy

0.8004444444444445

In [9]:
predictions = []
for image_logits in logits:
    summed_logits = image_logits.sum(dim=0)
    
    max_vote_key = summed_logits.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

logit_sum_accuracy = balanced_accuracy_score(labels, predictions)
logit_sum_accuracy

0.8226666666666667

In [10]:
predictions = []
for image_logits in logits:
    summed_probs = image_logits.softmax(dim=1).sum(dim=0)
    
    max_vote_key = summed_probs.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
prob_sum_accuracy

0.8093333333333333

In [11]:
predictions = []
for image_logits in logits:
    summed_log_probs = torch.log(image_logits.softmax(dim=1)).sum(dim=0)
    
    max_vote_key = summed_log_probs.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

log_prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
log_prob_sum_accuracy

0.8226666666666667

In [12]:
predictions = []
for image_logits in logits:
    summed_one_minus_log_probs = torch.log(1 - image_logits.softmax(dim=1)).sum(dim=0)
    
    min_vote_key = summed_one_minus_log_probs.argmin().cpu().item()
    predictions.append(integer_to_label[min_vote_key])

one_minus_log_prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
one_minus_log_prob_sum_accuracy

0.8173333333333332

In [13]:
predictions = []
for i, image_logits in enumerate(logits):
    summed_log_probs = torch.log(image_logits.softmax(dim=1)).sum(dim=0)
    summed_one_minus_log_probs = torch.log(1 - image_logits.softmax(dim=1)).sum(dim=0)
    if summed_log_probs.argmax().cpu().item() != summed_one_minus_log_probs.argmin().cpu().item():
        print(i, labels[i], integer_to_label[summed_log_probs.argmax().cpu().item()], integer_to_label[summed_one_minus_log_probs.argmin().cpu().item()], summed_log_probs, summed_one_minus_log_probs)


1 LGSC HGSC CC tensor([-180.7725, -224.9261, -264.5779, -323.4103, -523.8602],
       device='cuda:0') tensor([-62.1418, -62.2408, -41.1723, -18.8171,  -2.4730], device='cuda:0')
12 HGSC HGSC LGSC tensor([-223.7321, -432.4293, -239.4981, -246.8518, -438.3141],
       device='cuda:0') tensor([-64.5037,  -7.0144, -49.5736, -71.5718, -11.0424], device='cuda:0')
16 HGSC HGSC CC tensor([-197.0083, -197.3605, -269.5479, -508.6062, -547.8976],
       device='cuda:0') tensor([-77.7349, -94.7483, -39.6360,  -4.0842,  -2.4202], device='cuda:0')
79 HGSC HGSC LGSC tensor([ -59.4774, -114.8453,  -97.0722,  -62.9548, -165.7855],
       device='cuda:0') tensor([-18.8099,  -6.5171,  -7.9103, -24.9382,  -1.4559], device='cuda:0')
105 EC HGSC EC tensor([-142.0571, -155.0600, -174.7018, -201.5209, -240.1484],
       device='cuda:0') tensor([-26.5934, -27.9430, -28.6421, -15.4968, -14.0066], device='cuda:0')


In [14]:
from collections import defaultdict

def borda_count_winner(image_logits):
    num_classes = image_logits.size(1)
    borda_scores = defaultdict(int)

    # Rank each class for each image_logits and assign Borda points
    for logits in image_logits:
        # Get ranks (in descending order of logits)
        ranks = torch.argsort(logits, descending=True)

        # Assign Borda points (highest rank gets num_classes - 1 points, next gets num_classes - 2, ...)
        for rank, class_index in enumerate(ranks):
            borda_scores[class_index.item()] += num_classes - 1 - rank

    # Find the class with the highest total Borda score
    borda_winner = max(borda_scores, key=borda_scores.get)
    return borda_winner

predictions = []
for image_logits in logits:
    borda_winner = borda_count_winner(image_logits)
    predictions.append(integer_to_label[borda_winner])
    
borda_accuracy = balanced_accuracy_score(labels, predictions)
borda_accuracy

0.7915555555555555

In [15]:
def instant_runoff_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    active_candidates = set(range(num_classes))

    while True:
        # Count the first-preference votes for each candidate
        first_pref_counts = torch.zeros(num_classes)
        for logits in image_logits:
            for rank in torch.argsort(logits, descending=True):
                if rank.item() in active_candidates:
                    first_pref_counts[rank] += 1
                    break

        # Check if any candidate has more than 50% of the votes
        if torch.any(first_pref_counts > num_voters / 2):
            return torch.argmax(first_pref_counts).item()

        # Find the candidate with the fewest votes among active candidates
        active_candidates_votes = first_pref_counts[list(active_candidates)]
        min_votes, min_index = torch.min(active_candidates_votes, 0)
        min_candidate = list(active_candidates)[min_index.item()]

        # Eliminate the candidate with the fewest votes
        active_candidates.remove(min_candidate)
        
predictions = []
for image_logits in logits:
    irv_winner = instant_runoff_winner(image_logits)
    predictions.append(integer_to_label[irv_winner])
    
instant_runoff_accuracy = balanced_accuracy_score(labels, predictions)
instant_runoff_accuracy

0.804888888888889

In [16]:
def calculate_minimax_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    max_regrets = torch.zeros(num_classes)

    # Perform pairwise comparisons between all classes
    for i in range(num_classes):
        for j in range(i + 1, num_classes):
            # Count how many voters prefer class i over class j and vice versa
            votes_for_i = torch.sum(image_logits[:, i] > image_logits[:, j])
            votes_for_j = num_voters - votes_for_i

            # Update the maximum regret for each class
            max_regrets[i] = max(max_regrets[i], votes_for_j)
            max_regrets[j] = max(max_regrets[j], votes_for_i)

    # The winner is the class with the smallest maximum regret
    return torch.argmin(max_regrets).item()
        
predictions = []
for image_logits in logits:
    minimax_winner = calculate_minimax_winner(image_logits)
    predictions.append(integer_to_label[minimax_winner])
    
minimax_accuracy = balanced_accuracy_score(labels, predictions)
minimax_accuracy

0.8137777777777778

In [17]:
from collections import defaultdict

def find_winner(graph, source, visited):
    """Helper function to find the winner in the graph."""
    if source not in graph:
        return False

    visited.add(source)
    for target in graph[source]:
        if target not in visited and find_winner(graph, target, visited):
            return True
    visited.remove(source)
    return False

def ranked_pairs_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    margins = defaultdict(int)

    # Perform pairwise comparisons
    for i in range(num_classes):
        for j in range(num_classes):
            if i != j:
                votes_for_i = torch.sum(image_logits[:, i] > image_logits[:, j])
                votes_for_j = num_voters - votes_for_i
                margins[(i, j)] = votes_for_i - votes_for_j

    # Sort pairs by margin of victory
    sorted_pairs = sorted(margins, key=margins.get, reverse=True)

    # Initialize graph for locking pairs
    graph = {i: set() for i in range(num_classes)}
    for pair in sorted_pairs:
        winner, loser = pair
        graph[winner].add(loser)
        visited = set()

        # Check for cycle
        if find_winner(graph, loser, visited):
            graph[winner].remove(loser)

    # Determine the winner
    for i in range(num_classes):
        if not any(i in targets for targets in graph.values()):
            return i

    # Fallback: Choose the class with the highest total votes if no clear winner is found
    total_votes = torch.sum(image_logits, axis=0)
    return torch.argmax(total_votes).item()
        
predictions = []
for image_logits in logits:
    rp_winner = ranked_pairs_winner(image_logits)
    predictions.append(integer_to_label[rp_winner])
    
rp_accuracy = balanced_accuracy_score(labels, predictions)
rp_accuracy

0.8226666666666667

In [18]:
from collections import defaultdict

def star_voting_winner(image_logits):
    num_voters, num_classes = image_logits.shape

    # Step 1: Sum the scores for each candidate
    total_scores = torch.sum(image_logits, axis=0)

    # Step 2: Find the two candidates with the highest total scores
    top_two = torch.topk(total_scores, 2).indices

    # Step 3: Runoff between the top two candidates
    first_choice_votes = torch.sum(image_logits[:, top_two[0]] > image_logits[:, top_two[1]])
    second_choice_votes = num_voters - first_choice_votes

    # Determine the winner
    if first_choice_votes > second_choice_votes:
        return top_two[0].item()
    else:
        return top_two[1].item()
        
predictions = []
for image_logits in logits:
    star_winner = star_voting_winner(image_logits)
    predictions.append(integer_to_label[star_winner])
    
star_accuracy = balanced_accuracy_score(labels, predictions)
star_accuracy

0.8226666666666667